In [1]:
import pandas as pd
import ast
import json
import re

In [12]:
df = df = pd.read_csv('./mazda_data.csv', sep=',', header=0)
df


,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01 00:09:42,city0504,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""mingchaoliu""...",111.243.121.95,2020-12-01 00:04:07,ptt
1,2,https://www.ptt.cc/bbs/car/M.1606791807.A.CA7....,[新聞]預計明年現身，ToyotaRAV4將推全新動力！,2020-12-01 11:03:24,yamatobar,car,原文連結：\nhttps://auto.ltn.com.tw/news/16610/3\n原...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""XXXXBANG"", ""...",1.171.168.195,2020-12-02 00:04:03,ptt
2,3,https://www.ptt.cc/bbs/car/M.1606791901.A.39C....,[情報]新世代Mazda2/CX-3有望沿用Yaris平台,2020-12-01 11:04:57,oppoR20,car,新一代Mazda 2有望直接沿用Yaris平台，CX3也可能直接辦理！\nhttps://w...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""wang960615"",...",140.125.222.17,2020-12-02 00:04:03,ptt
3,4,https://www.ptt.cc/bbs/car/M.1606792933.A.D92....,Re:[情報]新世代Mazda2/CX-3有望沿用Yaris平台,2020-12-01 11:22:11,jason89514,car,這是依照之前 Mazda Q3 財報會上所公佈的資訊\nhttps://www.carstu...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""JOHN2188"", ""...",114.43.130.93,2020-12-02 00:04:03,ptt
4,5,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 11:55:33,jerrysuper,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""XXXXBANG"", ""...",111.251.90.133,2020-12-02 00:04:03,ptt
...,...,...,...,...,...,...,...,...,...,...,...
3501,3502,https://www.ptt.cc/bbs/CarShop/M.1674922265.A....,[購車]全新MazdaCX3020SCarbonEdition,2023-01-29 00:11:03,chaoyoyo,CarShop,車輛狀況：2023 全新\n\n車輛品牌：Mazda\n\n車款型式：CX30 20S Ca...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""a7426891"", ""...",115.43.31.123,2023-01-29 01:03:07,ptt
3502,3503,https://www.ptt.cc/bbs/CarShop/M.1675004691.A....,[購車]MazdaCX-520SPremiumSE,2023-01-29 23:04:49,Haskell,CarShop,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：CX-5 20S Premium...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""UPIO"", ""cmtC...",36.228.84.16,2023-01-30 01:03:03,ptt
3503,3504,https://www.ptt.cc/bbs/CarShop/M.1675041873.A....,[購車]2023MazdaCX-520SPremiumSE,2023-01-30 09:24:31,UPIO,CarShop,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：2023 Mazda CX-5 ...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""rain062811"",...",122.116.29.47,2023-01-31 01:03:07,ptt
3504,3505,https://www.ptt.cc/bbs/CarShop/M.1675050698.A....,[購車]Mazda35D2023年式20SSignature/Prem,2023-01-30 11:51:36,zx6226880,CarShop,車輛狀況：全新\n \n車輛品牌：Mazda3\n \n車款型式：Mazda3 5D 202...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""s77329"", ""cm...",223.138.72.252,2023-01-31 01:03:07,ptt


In [13]:
def check(_str):
    if (_str == None):
        return False
    elif ('[{"cmtStatus":' in _str):
        return True
    else:
        return False

In [14]:
def to_string(_str):
    tmp = str(_str)
    match = re.search(r'(?s:.*)(\, \{\"+[\s\S]*)', tmp)
    if (match == None):
        return tmp
    try:
        if (str(match.group(1))[-1] == "]"):
            # tmp = tmp.replace('“', '"')
            return tmp
        else:
            a = tmp.strip(str(match.group(1)))
            # a = a.replace('“', '"')
            return a+"]"
    except:
        print(tmp[:-20])

In [15]:
## 轉字串並處理格式
df['artComment'] = df['artComment'].apply(lambda x: to_string(x))
## string to list
df['artComment'] = df['artComment'].apply(lambda x: ast.literal_eval(x) if check(x) else x)

In [16]:
df.loc[0]['artComment'][0]

{'cmtStatus': '→',
 'cmtPoster': 'mingchaoliu',
 'cmtContent': ':要看樓主住哪或開車活動範圍在哪？西部是還好啦',
 'cmtDate': '2020-12-01 00:19:00'}

In [17]:
def concat_string(_list):
    if (_list == None):
        return _list
    else:
        if (type(_list) == list):
                tmp = ""
                for i in _list:
                    try:
                        tmp += i['cmtContent']
                    except:
                        print(i)
                return tmp
        else:
            return _list

In [18]:
df['concat_comment'] = df['artComment'].apply(lambda x: concat_string(x))
df

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,concat_comment
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01 00:09:42,city0504,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,"[{'cmtStatus': '→', 'cmtPoster': 'mingchaoliu'...",111.243.121.95,2020-12-01 00:04:07,ptt,:要看樓主住哪或開車活動範圍在哪？西部是還好啦:一年是可以去保養廠幾次:3008快小改款了:...
1,2,https://www.ptt.cc/bbs/car/M.1606791807.A.CA7....,[新聞]預計明年現身，ToyotaRAV4將推全新動力！,2020-12-01 11:03:24,yamatobar,car,原文連結：\nhttps://auto.ltn.com.tw/news/16610/3\n原...,"[{'cmtStatus': '推', 'cmtPoster': 'XXXXBANG', '...",1.171.168.195,2020-12-02 00:04:03,ptt,:電動浴缸？:防水電動車:哈哈:原廠有打算出敞篷RAV4嗎?:RAV250h？？？:跟特斯拉...
2,3,https://www.ptt.cc/bbs/car/M.1606791901.A.39C....,[情報]新世代Mazda2/CX-3有望沿用Yaris平台,2020-12-01 11:04:57,oppoR20,car,新一代Mazda 2有望直接沿用Yaris平台，CX3也可能直接辦理！\nhttps://w...,"[{'cmtStatus': '推', 'cmtPoster': 'wang960615',...",140.125.222.17,2020-12-02 00:04:03,ptt,:北美的YARIS停產了:但是這篇內文的東西總感覺可能全球發售:畢竟馬二的銷量除了日本本土看...
3,4,https://www.ptt.cc/bbs/car/M.1606792933.A.D92....,Re:[情報]新世代Mazda2/CX-3有望沿用Yaris平台,2020-12-01 11:22:11,jason89514,car,這是依照之前 Mazda Q3 財報會上所公佈的資訊\nhttps://www.carstu...,"[{'cmtStatus': '推', 'cmtPoster': 'JOHN2188', '...",114.43.130.93,2020-12-02 00:04:03,ptt,:進口、最新1.5油電、LV2，很香啊
4,5,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 11:55:33,jerrysuper,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,"[{'cmtStatus': '推', 'cmtPoster': 'XXXXBANG', '...",111.251.90.133,2020-12-02 00:04:03,ptt,:反指標板真的狂風向反著吹:Kuga贏CRV:雨衣廠商連帶受惠:看車版以為RAV4銷量剩不到...
...,...,...,...,...,...,...,...,...,...,...,...,...
3501,3502,https://www.ptt.cc/bbs/CarShop/M.1674922265.A....,[購車]全新MazdaCX3020SCarbonEdition,2023-01-29 00:11:03,chaoyoyo,CarShop,車輛狀況：2023 全新\n\n車輛品牌：Mazda\n\n車款型式：CX30 20S Ca...,"[{'cmtStatus': '推', 'cmtPoster': 'a7426891', '...",115.43.31.123,2023-01-29 01:03:07,ptt,:鐵灰+1:+1:北部鐵灰+1:台南+1
3502,3503,https://www.ptt.cc/bbs/CarShop/M.1675004691.A....,[購車]MazdaCX-520SPremiumSE,2023-01-29 23:04:49,Haskell,CarShop,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：CX-5 20S Premium...,"[{'cmtStatus': '推', 'cmtPoster': 'UPIO', 'cmtC...",36.228.84.16,2023-01-30 01:03:03,ptt,:+1:+1:+1:+1
3503,3504,https://www.ptt.cc/bbs/CarShop/M.1675041873.A....,[購車]2023MazdaCX-520SPremiumSE,2023-01-30 09:24:31,UPIO,CarShop,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：2023 Mazda CX-5 ...,"[{'cmtStatus': '推', 'cmtPoster': 'rain062811',...",122.116.29.47,2023-01-31 01:03:07,ptt,:+1:+1:+1:+1:+1:+1:+1
3504,3505,https://www.ptt.cc/bbs/CarShop/M.1675050698.A....,[購車]Mazda35D2023年式20SSignature/Prem,2023-01-30 11:51:36,zx6226880,CarShop,車輛狀況：全新\n \n車輛品牌：Mazda3\n \n車款型式：Mazda3 5D 202...,"[{'cmtStatus': '推', 'cmtPoster': 's77329', 'cm...",223.138.72.252,2023-01-31 01:03:07,ptt,":高雄台南+1，謝謝:中部+1謝謝:北部+1:雙北+1:北部&宜蘭+1:高雄台南4D+1,謝..."


In [19]:
df['whole_art'] = df['artContent'] + df['concat_comment']

In [20]:
df

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,concat_comment,whole_art
0,1,https://www.ptt.cc/bbs/car/M.1606752584.A.175....,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020-12-01 00:09:42,city0504,car,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...,"[{'cmtStatus': '→', 'cmtPoster': 'mingchaoliu'...",111.243.121.95,2020-12-01 00:04:07,ptt,:要看樓主住哪或開車活動範圍在哪？西部是還好啦:一年是可以去保養廠幾次:3008快小改款了:...,原文連結：\nhttps://ctee.com.tw/lohas/car/378518.ht...
1,2,https://www.ptt.cc/bbs/car/M.1606791807.A.CA7....,[新聞]預計明年現身，ToyotaRAV4將推全新動力！,2020-12-01 11:03:24,yamatobar,car,原文連結：\nhttps://auto.ltn.com.tw/news/16610/3\n原...,"[{'cmtStatus': '推', 'cmtPoster': 'XXXXBANG', '...",1.171.168.195,2020-12-02 00:04:03,ptt,:電動浴缸？:防水電動車:哈哈:原廠有打算出敞篷RAV4嗎?:RAV250h？？？:跟特斯拉...,原文連結：\nhttps://auto.ltn.com.tw/news/16610/3\n原...
2,3,https://www.ptt.cc/bbs/car/M.1606791901.A.39C....,[情報]新世代Mazda2/CX-3有望沿用Yaris平台,2020-12-01 11:04:57,oppoR20,car,新一代Mazda 2有望直接沿用Yaris平台，CX3也可能直接辦理！\nhttps://w...,"[{'cmtStatus': '推', 'cmtPoster': 'wang960615',...",140.125.222.17,2020-12-02 00:04:03,ptt,:北美的YARIS停產了:但是這篇內文的東西總感覺可能全球發售:畢竟馬二的銷量除了日本本土看...,新一代Mazda 2有望直接沿用Yaris平台，CX3也可能直接辦理！\nhttps://w...
3,4,https://www.ptt.cc/bbs/car/M.1606792933.A.D92....,Re:[情報]新世代Mazda2/CX-3有望沿用Yaris平台,2020-12-01 11:22:11,jason89514,car,這是依照之前 Mazda Q3 財報會上所公佈的資訊\nhttps://www.carstu...,"[{'cmtStatus': '推', 'cmtPoster': 'JOHN2188', '...",114.43.130.93,2020-12-02 00:04:03,ptt,:進口、最新1.5油電、LV2，很香啊,這是依照之前 Mazda Q3 財報會上所公佈的資訊\nhttps://www.carstu...
4,5,https://www.ptt.cc/bbs/car/M.1606794935.A.AA7....,[情報]2020年11月份臺灣汽車市場銷售報告,2020-12-01 11:55:33,jerrysuper,car,新增小七車\nhttps://www.7car.tw/articles/read/70876...,"[{'cmtStatus': '推', 'cmtPoster': 'XXXXBANG', '...",111.251.90.133,2020-12-02 00:04:03,ptt,:反指標板真的狂風向反著吹:Kuga贏CRV:雨衣廠商連帶受惠:看車版以為RAV4銷量剩不到...,新增小七車\nhttps://www.7car.tw/articles/read/70876...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3501,3502,https://www.ptt.cc/bbs/CarShop/M.1674922265.A....,[購車]全新MazdaCX3020SCarbonEdition,2023-01-29 00:11:03,chaoyoyo,CarShop,車輛狀況：2023 全新\n\n車輛品牌：Mazda\n\n車款型式：CX30 20S Ca...,"[{'cmtStatus': '推', 'cmtPoster': 'a7426891', '...",115.43.31.123,2023-01-29 01:03:07,ptt,:鐵灰+1:+1:北部鐵灰+1:台南+1,車輛狀況：2023 全新\n\n車輛品牌：Mazda\n\n車款型式：CX30 20S Ca...
3502,3503,https://www.ptt.cc/bbs/CarShop/M.1675004691.A....,[購車]MazdaCX-520SPremiumSE,2023-01-29 23:04:49,Haskell,CarShop,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：CX-5 20S Premium...,"[{'cmtStatus': '推', 'cmtPoster': 'UPIO', 'cmtC...",36.228.84.16,2023-01-30 01:03:03,ptt,:+1:+1:+1:+1,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：CX-5 20S Premium...
3503,3504,https://www.ptt.cc/bbs/CarShop/M.1675041873.A....,[購車]2023MazdaCX-520SPremiumSE,2023-01-30 09:24:31,UPIO,CarShop,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：2023 Mazda CX-5 ...,"[{'cmtStatus': '推', 'cmtPoster': 'rain062811',...",122.116.29.47,2023-01-31 01:03:07,ptt,:+1:+1:+1:+1:+1:+1:+1,車輛狀況：全新\n\n車輛品牌：Mazda\n\n車款型式：2023 Mazda CX-5 ...
3504,3505,https://www.ptt.cc/bbs/CarShop/M.1675050698.A....,[購車]Mazda35D2023年式20SSignature/Prem,2023-01-30 11:51:36,zx6226880,CarShop,車輛狀況：全新\n \n車輛品牌：Mazda3\n \n車款型式：Mazda3 5D 202...,"[{'cmtStatus': '推', 'cmtPoster': 's77329', 'cm...",223.138.72.252,2023-01-31 01:03:07,ptt,":高雄台南+1，謝謝:中部+1謝謝:北部+1:雙北+1:北部&宜蘭+1:高雄台南4D+1,謝...",車輛狀況：全新\n \n車輛品牌：Mazda3\n \n車款型式：Mazda3 5D 202...


In [21]:
# df_1 = df[:5000]
# df_2 = df[5001:]

In [22]:
df.to_csv('./mazda_data_fix_1.csv', encoding='utf-8')
# df_2.to_csv('./toyota_data_fix_2.csv', encoding='utf-8')

- mobile
    - ["'\\n這單真的算不錯啊~期待新車開箱\\n'", " '\\n大大請問一下 是直接現金價嗎 還是有分期也可以這麼阿莎力\\n'", " '\\n請問 MIO 跟原廠可結合車機螢幕的差別是 ？實在不太想前面多太多裝置 ...\\n'", " '\\n想問樓主方便分享分期利率大概在？\\n'", " '\\n樓主可私訊 業代名字嗎🤣\\n'", " '\\n請問高雄有業務能以此單來賣的嗎？請私訊\\n'", " '\\nChandlerwu wrote:請問高雄有業務能以此...(恕刪)\\n高雄+1\\n'", " '\\n高雄+1...\\n'"]
- ptt
    - [{"cmtStatus": "推", "cmtPoster": "XXXXBANG", "cmtContent": ":反指標板真的狂風向反著吹", "cmtDate": "2020-12-01 11:57:00"}, {"cmtStatus": "推", "cmtPoster": "hito80686", "cmtContent": ":Kuga贏CRV", "cmtDate": "2020-12-01 11:57:00"}, {"cmtStatus": "推", "cmtPoster": "wonderlands", "cmtContent": ":雨衣廠商連帶受惠", "cmtDate": "2020-12-01 11:57:00"}, {"cmtStatus": "推", "cmtPoster": "js52666", "cmtContent": ":看車版以為RAV4銷量剩不到100台", "cmtDate": "2020-12-01 11:58:00"},
- Dcard
    - 買油電ALTIS啊，現在大電池保固8年16萬根本無需擔心電池問題，又省油省到爆，我自己再開平均油耗ㄧ公升都有跑到24公里
    - ["''"]